# Build a DNN with 20 hidden layer of 100 neurons each.Use He initialization and ELU activation function.


In [11]:
from tensorflow import keras
import tensorflow as tf
import numpy as np
import sklearn
import tensorboard
from sklearn import model_selection


keras.backend.clear_session()
tf.random.set_seed(42)
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
np.random.seed(42)
for _ in range(20):
    model.add(keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"))


c:\Users\ADMIN\Desktop\Jupyter Projects\env\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Using Nadam optimization and early stopping, train the network on the CIFAR10 dataset. You can load it as: keras.datasets.cifar10.Load_data(). The dataset is composed of 60000 32 x 32 pixel color images(50000 for training and 10000 for testing) with 10 classes, so you'll need a softmax output layer with 10 neurons. Remember to search for the right learning rate each time you change the models's architecture or hyperparameter.

In [12]:
model.add(keras.layers.Dense(10, activation="softmax"))

In [13]:
# here we use the Nadam optimizer with a learning rate of 5e-5, it is obtained after
# some trial and error, trying learning rates 1e-5, 3e-5, 1e-4, 3e-4, 1e-3, 3e-3 and 1e-2, and comparing their learning curves for 10 epochs each (using the TensorBoard callback, below).  
optimizer = keras.optimizers.Nadam(learning_rate=5e-5)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])


In [14]:
# Loading the CIFAR10 dataset. We also want to use Early stopping so we need a validation set.
# We use first 5000 images of the original training set as the  validation set.
import os
import pickle
import numpy as np

def load_cifar10_batch(batch_filename):
    with open(batch_filename, 'rb') as f:
        batch = pickle.load(f, encoding='latin1')
        data = batch['data']
        labels = batch['labels']
        data = data.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)
        return data, np.array(labels)

def load_cifar10(cifar10_dir):
    X_train_full, y_train_full = [], []
    for i in range(1, 6):
        batch_path = os.path.join(cifar10_dir, f'data_batch_{i}')
        data, labels = load_cifar10_batch(batch_path)
        X_train_full.append(data)
        y_train_full.append(labels)
    X_train_full = np.concatenate(X_train_full)
    y_train_full = np.concatenate(y_train_full)

    X_test, y_test = load_cifar10_batch(os.path.join(cifar10_dir, 'test_batch'))
    return (X_train_full, y_train_full), (X_test, y_test)

# 👉 Change this to where you extracted the files
cifar10_path = r"C:\Users\ADMIN\Downloads\cifar-10-python\cifar-10-batches-py"

(X_train_full, y_train_full), (X_test, y_test) = load_cifar10(cifar10_path)

# Same split logic as before
X_valid = X_train_full[:5000]
y_valid = y_train_full[:5000]
X_train = X_train_full[5000:]
y_train = y_train_full[5000:]


In [ ]:
# Here we create callbacks we need and train the model.

In [15]:
import os
from tensorflow import keras


early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_model.h5", save_best_only=True)
run_index = 1 # Increment every time you train the model.
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_{:03d}".format(run_index))
tesnsorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tesnsorboard_cb]

In [6]:
%tensorboard --logdir=./my_cifar10_logs --port=6006

UsageError: Line magic function `%tensorboard` not found.


In [16]:
model.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=callbacks)

Epoch 1/100
1404/1407 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.1284 - loss: 10.9341

1407/1407 ━━━━━━━━━━━━━━━━━━━━ 27s 15ms/step - accuracy: 0.1285 - loss: 10.9157 - val_accuracy: 0.2226 - val_loss: 2.1710
Epoch 2/100
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.2247 - loss: 2.1210

1407/1407 ━━━━━━━━━━━━━━━━━━━━ 21s 15ms/step - accuracy: 0.2247 - loss: 2.1209 - val_accuracy: 0.2668 - val_loss: 2.0167
Epoch 3/100
1404/1407 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.2699 - loss: 1.9793

1407/1407 ━━━━━━━━━━━━━━━━━━━━ 18s 13ms/step - accuracy: 0.2699 - loss: 1.9792 - val_accuracy: 0.2952 - val_loss: 1.9232
Epoch 4/100
1404/1407 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3067 - loss: 1.9055

1407/1407 ━━━━━━━━━━━━━━━━━━━━ 14s 10ms/step - accuracy: 0.3068 - loss: 1.9054 - val_accuracy: 0.3236 - val_loss: 1.8733
Epoch 5/100
1404/1407 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3288 - loss: 1.8401

...

1407/1407 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.4372 - loss: 1.5751 - val_accuracy: 0.4108 - val_loss: 1.6278
Epoch 13/100
1405/1407 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4434 - loss: 1.5538

1407/1407 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.4434 - loss: 1.5537 - val_accuracy: 0.4278 - val_loss: 1.6004
Epoch 14/100
1403/1407 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4500 - loss: 1.5367

1407/1407 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.4500 - loss: 1.5367 - val_accuracy: 0.4266 - val_loss: 1.5985
Epoch 15/100
1402/1407 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4553 - loss: 1.5207

1407/1407 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.4553 - loss: 1.5207 - val_accuracy: 0.4320 - val_loss: 1.5866
Epoch 16/100
1403/1407 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4570 - loss: 1.5023

1407/1407 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.4570 - loss: 1.5023 - val_accuracy: 0.4382 - val_loss: 1.5710
Epoch 17/100
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.4631 - loss: 1.4906 - val_accuracy: 0.4352 - val_loss: 1.5827
Epoch 18/100
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.4697 - loss: 1.4736 - val_accuracy: 0.4394 - val_loss: 1.5824
Epoch 19/100
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.4760 - loss: 1.4567 - val_accuracy: 0.4394 - val_loss: 1.5821
Epoch 20/100
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.4819 - loss: 1.4436 - val_accuracy: 0.4330 - val_loss: 1.5831
Epoch 21/100
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.4861 - loss: 1.4326 - val_accuracy: 0.4394 - val_loss: 1.5820
Epoch 22/100
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.4901 - loss: 1.4206 - val_accuracy: 0.4428 - val_loss: 1.5983
Epoch 23/100
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.4961 - loss

In [17]:
model = keras.models.load_model("my_cifar10_model.h5")
model.evaluate(X_valid, y_valid)

157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4412 - loss: 1.5728


[1.5710365772247314, 0.4381999969482422]

# Now try adding Batch Normalization and compare the learning curves:Is it converging faster than before? Does it produce a better model? How does it affect
# training speed?


In [ ]:
# To answer this part, we use the same code as above except some changes:
#  *) adding a BN layer after every Dense layer(before the activation fn), except for the output layer. Also added a BN layer 
# before the first hidden layer.

# *) Changed the learning rate to 5e-4.
# *) Renamed the run directions to run_bn_* and the model file name to my_cifar10_bn_model.h5


In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
model.add(keras.layers.BatchNormalization())
for _ in range(20):
    model.add(keras.layers.Dense(100, kernel_initializer="he_normal"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("elu"))
model.add(keras.layers.Dense(10, activation="softmax"))

optimizer = keras.optimizers.Nadam(learning_rate=5e-4)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_bn_model.h5", save_best_only=True)
run_index = 1 # Increment every time you train the model.
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_bn_{:03d}".format(run_index))
tesnsorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tesnsorboard_cb]

model.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=callbacks)

model = keras.models.load_model("my_cifar10_bn_model.h5")
model.evaluate(X_valid, y_valid)


Epoch 1/100
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2976 - loss: 1.9675

1407/1407 ━━━━━━━━━━━━━━━━━━━━ 30s 11ms/step - accuracy: 0.2976 - loss: 1.9674 - val_accuracy: 0.3948 - val_loss: 1.6898
Epoch 2/100
1404/1407 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4038 - loss: 1.6828

1407/1407 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.4038 - loss: 1.6828 - val_accuracy: 0.4334 - val_loss: 1.6006
Epoch 3/100
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4383 - loss: 1.5886

1407/1407 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - accuracy: 0.4383 - loss: 1.5886 - val_accuracy: 0.4464 - val_loss: 1.5590
Epoch 4/100
1402/1407 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4616 - loss: 1.5180

1407/1407 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.4616 - loss: 1.5180 - val_accuracy: 0.4550 - val_loss: 1.5346
Epoch 5/100
1404/1407 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4873 - loss: 1.4559

1407/1407 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - accuracy: 0.4873 - loss: 1.4559 - val_accuracy: 0.4648 - val_loss: 1.5184
Epoch 6/100
1403/1407 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5068 - loss: 1.4000

1407/1407 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - accuracy: 0.5068 - loss: 1.4000 - val_accuracy: 0.4698 - val_loss: 1.5043
Epoch 7/100
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - accuracy: 0.5299 - loss: 1.3455 - val_accuracy: 0.4636 - val_loss: 1.5246
Epoch 8/100
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - accuracy: 0.5478 - loss: 1.2971 - val_accuracy: 0.4720 - val_loss: 1.5216
Epoch 9/100
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - accuracy: 0.5661 - loss: 1.2508 - val_accuracy: 0.4642 - val_loss: 1.5659
Epoch 10/100
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - accuracy: 0.5830 - loss: 1.2124 - val_accuracy: 0.4636 - val_loss: 1.5907
Epoch 11/100
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 16s 11ms/step - accuracy: 0.5931 - loss: 1.1734 - val_accuracy: 0.4640 - val_loss: 1.6209
Epoch 12/100
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - accuracy: 0.6074 - loss: 1.1387 - val_accuracy: 0.4562 - val_loss: 1.6718
Epoch 13/100
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - accuracy: 0.6185 -

157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4703 - loss: 1.5168


[1.504280686378479, 0.4697999954223633]

# Excr:
# Try replacing the Batch Normalization with SELU, and make the necessary adustments to ensure that the network self-normalizes(i.e, standardize the input features, 
# use LeCun normal initialization, make sure the DDN contains only a sequence of dense layer, etc).


In [24]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32,32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100, kernel_initializer="lecun_normal", activation="selu"))
model.add(keras.layers.Dense(10, activation="softmax"))

optimizer = keras.optimizers.Nadam(learning_rate=7e-4)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_selu_model.h5", save_best_only=True)
run_index = 1 # Increment every time you train the model.
tesnsorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tesnsorboard_cb]

X_means = X_train.mean(axis=0)
X_stds = X_train.std(axis=0)
X_train_scaled = (X_train - X_means) / X_stds
X_valid_scaled = (X_valid - X_means) / X_stds
X_test_scaled = (X_test - X_means) / X_stds

model.fit(X_train_scaled, y_train, epochs=100, validation_data=(X_valid_scaled, y_valid), callbacks=callbacks)
model = keras.models.load_model("my_cifar10_selu_model.h5")
model.evaluate(X_valid_scaled, y_valid) 

Epoch 1/100
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.2787 - loss: 2.0281

1407/1407 ━━━━━━━━━━━━━━━━━━━━ 29s 15ms/step - accuracy: 0.2787 - loss: 2.0280 - val_accuracy: 0.3432 - val_loss: 1.7810
Epoch 2/100
1405/1407 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3840 - loss: 1.7252

1407/1407 ━━━━━━━━━━━━━━━━━━━━ 20s 14ms/step - accuracy: 0.3840 - loss: 1.7251 - val_accuracy: 0.4098 - val_loss: 1.6911
Epoch 3/100
1406/1407 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4248 - loss: 1.6197

1407/1407 ━━━━━━━━━━━━━━━━━━━━ 20s 14ms/step - accuracy: 0.4248 - loss: 1.6196 - val_accuracy: 0.4366 - val_loss: 1.6203
Epoch 4/100
1404/1407 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4533 - loss: 1.5477

1407/1407 ━━━━━━━━━━━━━━━━━━━━ 21s 15ms/step - accuracy: 0.4533 - loss: 1.5477 - val_accuracy: 0.4486 - val_loss: 1.5609
Epoch 5/100
1405/1407 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4813 - loss: 1.4853

...

1407/1407 ━━━━━━━━━━━━━━━━━━━━ 21s 15ms/step - accuracy: 0.5000 - loss: 1.4356 - val_accuracy: 0.4834 - val_loss: 1.5361
Epoch 7/100
1404/1407 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5135 - loss: 1.3981

1407/1407 ━━━━━━━━━━━━━━━━━━━━ 21s 15ms/step - accuracy: 0.5135 - loss: 1.3980 - val_accuracy: 0.4796 - val_loss: 1.5271
Epoch 8/100
1403/1407 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5307 - loss: 1.3541

1407/1407 ━━━━━━━━━━━━━━━━━━━━ 20s 14ms/step - accuracy: 0.5307 - loss: 1.3541 - val_accuracy: 0.4782 - val_loss: 1.5235
Epoch 9/100
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 21s 15ms/step - accuracy: 0.5400 - loss: 1.3218 - val_accuracy: 0.4958 - val_loss: 1.5269
Epoch 10/100
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 18s 13ms/step - accuracy: 0.5630 - loss: 1.2823 - val_accuracy: 0.4906 - val_loss: 1.5388
Epoch 11/100
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.5680 - loss: 1.2585 - val_accuracy: 0.4904 - val_loss: 1.5259
Epoch 12/100
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 16s 11ms/step - accuracy: 0.5779 - loss: 1.2311 - val_accuracy: 0.4932 - val_loss: 1.5506
Epoch 13/100
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 14s 10ms/step - accuracy: 0.5956 - loss: 1.1914 - val_accuracy: 0.4926 - val_loss: 1.5979
Epoch 14/100
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 14s 10ms/step - accuracy: 0.6001 - loss: 1.1724 - val_accuracy: 0.4944 - val_loss: 1.5724
Epoch 15/100
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 17s 12ms/step - accuracy: 0.6069

157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.4851 - loss: 1.5193


[1.5234951972961426, 0.4781999886035919]

In [25]:
model = keras.models.load_model("my_cifar10_selu_model.h5")
model.evaluate(X_valid_scaled, y_valid)

157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4851 - loss: 1.5193


[1.5234951972961426, 0.4781999886035919]

In [26]:
# Try regularizing the model with alpha dropout.Then, without retraining your model
# see if you can achieve  better accuracy using MC dropout.
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100, kernel_initializer="lecun_normal", activation="selu"))
model.add(keras.layers.AlphaDropout(rate=0.1))
model.add(keras.layers.Dense(10, activation="softmax"))

optimizer = keras.optimizers.Nadam(learning_rate=5e-4)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_alpha_dropout_model.h5", save_best_only=True)
run_index = 1 # Increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar1_logs", "run_alpha_dropout{:03d}".format(run_index))
tesnsorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tesnsorboard_cb]

X_means = X_train.mean(axis=0)
X_stds = X_train.std(axis=0)
X_train_scaled = (X_train - X_means) / X_stds
X_test_scaled = (X_test - X_means) / X_stds

model.fit(X_train_scaled, y_train, epochs=100, validation_data=(X_valid_scaled, y_valid), callbacks=callbacks)

model = keras.models.load_model("my_cifar10_alpha_dropout_model.h5")
model.evaluate(X_valid_scaled, y_valid)

Epoch 1/100
1403/1407 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2833 - loss: 2.0468

1407/1407 ━━━━━━━━━━━━━━━━━━━━ 23s 11ms/step - accuracy: 0.2834 - loss: 2.0462 - val_accuracy: 0.4018 - val_loss: 1.6902
Epoch 2/100
1406/1407 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3968 - loss: 1.6932

1407/1407 ━━━━━━━━━━━━━━━━━━━━ 16s 11ms/step - accuracy: 0.3968 - loss: 1.6931 - val_accuracy: 0.4186 - val_loss: 1.6637
Epoch 3/100
1406/1407 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4359 - loss: 1.5940

1407/1407 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - accuracy: 0.4359 - loss: 1.5940 - val_accuracy: 0.4448 - val_loss: 1.6289
Epoch 4/100
1406/1407 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4649 - loss: 1.5213

1407/1407 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - accuracy: 0.4649 - loss: 1.5213 - val_accuracy: 0.4566 - val_loss: 1.5827
Epoch 5/100
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - accuracy: 0.4918 - loss: 1.4548 - val_accuracy: 0.4754 - val_loss: 1.5849
Epoch 6/100
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - accuracy: 0.5114 - loss: 1.4097 - val_accuracy: 0.4754 - val_loss: 1.5891
Epoch 7/100
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.5269 - loss: 1.3590 - val_accuracy: 0.4814 - val_loss: 1.6057
Epoch 8/100
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 17s 12ms/step - accuracy: 0.5437 - loss: 1.3199 - val_accuracy: 0.4854 - val_loss: 1.6456
Epoch 9/100
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - accuracy: 0.5558 - loss: 1.2818 - val_accuracy: 0.4900 - val_loss: 1.5906
Epoch 10/100
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - accuracy: 0.5680 - loss: 1.2516 - val_accuracy: 0.4836 - val_loss: 1.6308
Epoch 11/100
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.5842 - lo

157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4589 - loss: 1.5697


[1.5826939344406128, 0.45660001039505005]

In [27]:
# Using MC Dropout 
class MCAlphaDropout(keras.layers.AlphaDropout):
    def call(self, inputs):
        return super().call(inputs, training=True)

In [28]:
# This is identical to the one that just trained (with the same weights),but with MCAlphaDropout dropout layers instead of AlphaDropout layers.

mc_model = keras.models.Sequential([
    MCAlphaDropout(layer.rate) if isinstance(layer, keras.layers.AlphaDropout) else layer
    for layer in model.layers
])

In [29]:
# Here we add a couple of utility fns: The first will run the model many times (10 by default) and it will return the mean predicted class probabilities.
# The second will use these mean probabilites to predict the most likely class for each instance:
def mc_dropout_predict_probas(mc_model, X, n_samples=10):
    Y_probas = [mc_model.predict(X) for smaple in range(n_samples)]
    return np.mean(Y_probas, axis=0)

def mc_dropout_predict_classes(mc_model, X, n_samples=10):
    Y_probas = mc_dropout_predict_probas(mc_model, X, n_samples)
    return np.argmax(Y_probas, axis=1)

In [31]:
# Now we make predictions for all instance in the validation set, and compute the accuracy.

keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

y_pred = mc_dropout_predict_classes(mc_model, X_valid_scaled)
accuracy = np.mean(y_pred == y_valid)
accuracy

157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


np.float64(0.4566)

In [ ]:
# Excer: Retain your model using 1cycle scheduling and see if it improves training speed and model accuracy.


In [50]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))

for _ in range(20):
    model.add(keras.layers.Dense(100, kernel_initializer="lecun_normal", activation="selu"))

model.add(keras.layers.AlphaDropout(rate=0.1))
model.add(keras.layers.Dense(10, activation="softmax"))

optimizer = keras.optimizers.SGD(learning_rate=1e-3)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,metrics=["accuracy"])



c:\Users\ADMIN\Desktop\Jupyter Projects\env\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [51]:
import matplotlib.pyplot as plt
import math
K = keras.backend

class ExponentialLearningRate(keras.callbacks.Callback):
    def __init__(self, factor):
        self.factor = factor
        self.rates = []
        self.losses = []
    def on_batch_end(self, batch, logs):
        self.rates.append(K.get_value(self.model.optimizer.learning_rate))    
        self.losses.append(logs["loss"])
        K.set_value(self.model.optimizer.learning_rate, self.model.optimizer.learning_rate * self.factor)

def find_learning_rate(model, X, y, epochs=1, batch_size=32, min_rate=1e-5, max_rate=10):
    init_weights = model.get_weights()
    iterations = math.ceil(len(X) / batch_size) * epochs
    factor = np.exp(np.log(max_rate / min_rate) / iterations)
    init_lr = K.get_value(model.optimizer.learning_rate)
    K.set_value(model.optimizer.learning_rate, min_rate)
    exp_lr = ExponentialLearningRate(factor)
    model.fit(X, y, epochs=epochs, batch_size=batch_size, callbacks=[exp_lr])
    K.set_value(model.optimizer.learning_rate, init_lr)
    model.set_weights(init_weights)
    return exp_lr.rates, exp_lr.losses

def plot_lr_vs_loss(rates, losses):
    plt.plot(rates, losses)
    plt.gca().set_xscale('log')
    plt.hlines(min(losses), min(rates), max(rates))
    plt.axis([min(rates), max(rates), min(losses), (losses[0] + min(losses)) / 2])
    plt.xlabel("Learning rate")
    plt.ylabel("Loss")


In [35]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100, kernel_initializer="lecun_normal", activation="selu"))

model.add(keras.layers.AlphaDropout(rate=0.1))
model.add(keras.layers.Dense(10, activation="softmax"))

optimizer = keras.optimizers.SGD(learning_rate=1e-2)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

c:\Users\ADMIN\Desktop\Jupyter Projects\env\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [37]:
class OneCycleScheduler(keras.callbacks.Callback):
    def __init__(self, iterations, max_rate, start_rate=None, last_iterations=None, last_rate=None):
        self.iterations = iterations
        self.max_rate = max_rate
        self.start_rate = start_rate or max_rate / 10
        self.last_iterations = last_iterations or iterations // 10 + 1
        self.half_iteration = (iterations - self.last_iterations) // 2
        self.last_rate = last_rate or self.start_rate / 1000
        self.iteration = 0

    def _interpolate(self, iter1, iter2, rate1, rate2):
        return ((rate2 - rate1) * (self.iteration - iter1) / (iter2 - iter1) + rate1)

    def on_batch(self, batch, logs):
        if self.iteration < self.half_iteration:
            rate = self._interpolate(0,self.half_iteration, self.start_rate, self.max_rate)
        elif self.iteration < 2 * self.half_iteration:
            rate = self._interpolate(self.half_iteration, 2 * self.half_iteration, self.max_rate, self.start_rate)

        else:
            rate = self._interpolate(2 * self.half_iteration, self.iterations, self.start_rate, self.last_rate) 
        
        self.iteration += 1
        K.set_value(self.model.optimizer.learning_rate, rate)


In [40]:
import math
batch_size =32
n_epochs = 15
onecycle = OneCycleScheduler(math.ceil(len(X_train_scaled) / batch_size) * n_epochs, max_rate=0.05)
history = model.fit(X_train_scaled, y_train, epochs=n_epochs, batch_size=batch_size, validation_data=(X_valid_scaled, y_valid), callbacks=[onecycle])



Epoch 1/15
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.2747 - loss: 2.0667 - val_accuracy: 0.3950 - val_loss: 1.7210
Epoch 2/15
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.4063 - loss: 1.6648 - val_accuracy: 0.4148 - val_loss: 1.6822
Epoch 3/15
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.4489 - loss: 1.5504 - val_accuracy: 0.4284 - val_loss: 1.6815
Epoch 4/15
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.4782 - loss: 1.4694 - val_accuracy: 0.4320 - val_loss: 1.6574
Epoch 5/15
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.5002 - loss: 1.4055 - val_accuracy: 0.4470 - val_loss: 1.6633
Epoch 6/15
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.5196 - loss: 1.3483 - val_accuracy: 0.4642 - val_loss: 1.6678
Epoch 7/15
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.5367 - loss: 1.3006 - val_accuracy: 0.4768 - val_loss: 1.6176
Epoch 8/15
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.5562 - loss: 1.2529 - 